In [17]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [18]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present.
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)

In [19]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    # df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

# Lặp Jacobi

In [20]:
def check_dominance(A: np.ndarray) -> int:
    """
    Returns:
      1 if A is row-dominant only,
      2 if A is column-dominant only,
      3 if both,
      0 if neither.
    """
    n = A.shape[0]
    row_dom = all(abs(A[i,i]) > np.sum(np.abs(A[i,:])) - abs(A[i,i]) for i in range(n))
    col_dom = all(abs(A[j,j]) > np.sum(np.abs(A[:,j])) - abs(A[j,j]) for j in range(n))
    if row_dom and col_dom:
        return 3
    if row_dom:
        return 1
    if col_dom:
        return 2
    return 0

## 1. Ma trận chéo trội Hàng

### Thuật toán

In [21]:
def convert_to_iteration(A: np.ndarray, B: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts the system Ax = B into the iterative form x_new = C x + D.
    """

    # Step 1: Build diagonal matrix T where T[i,i] = 1 / A[i,i]
    T = np.diag(1.0 / np.diag(A))
    
    # Step 2: Compute C = I - T * A
    n = A.shape[0]
    C = np.eye(n) - T.dot(A)
    
    # Step 3: Compute D = T * B
    D = T.dot(B)
    
    return C, D

In [22]:
def fixed_point_matrix_iteration(
    A: np.ndarray,
    C: np.ndarray,
    D: np.ndarray,
    x0: Union[np.ndarray, list],
    domiType: int,
    eps: float,
    eta: float
) -> pd.DataFrame:
    """
    Performs fixed-point iteration x_{k+1}=C x_k + D and returns history with errors.
    Convergence based on:
      - exact error if eps is provided (||x_new - x_old|| <= eps*(1-q)/(α*q))
      - relative error if eta is provided (||x_new-x_old||/||x_new|| <= eta*(1-q)/(α*q))

    Parameters:
    - C, D: iteration matrix and constant
    - x0: initial guess
    - domiType: 1 or 3 (row), 2 (column)
    - eps: absolute tolerance (use exact error)
    - eta: relative tolerance (use ||Δx||/||x_new||)
    """
    # Validate tolerance inputs
    if (eps is None) == (eta is None):
        raise ValueError("Specify exactly one of eps (exact) or eta (relative)")

    # Set norms and α, q based on dominance type
    if domiType in (1, 3):
        vec_norm = lambda x: np.max(np.abs(x))
        q = np.max(np.sum(np.abs(C), axis=1))
        alpha = 1.0
    else:
        raise ValueError("domiType must be 1 or 3 for row-dominant iteration")

    # Convergence threshold
    tol = (eps if eps is not None else eta) * (1 - q) / (alpha * q)
    print(f"Shrinking value: {q:.12f}, Additional value: {alpha:.12f}, Threshold: {tol:.12f}")

    # Initialize
    x_old = np.array(x0, dtype=float).flatten()
    history = [x_old.copy()]
    errors = [np.nan]

    # Iteration loop
    while True:
        x_new = C.dot(x_old) + D
        if eps is not None:
            err = vec_norm(x_new - x_old)
        else:
            err = vec_norm(x_new - x_old) / vec_norm(x_new)

        # Prepare DataFrame for display
        history.append(x_new.copy())
        errors.append(err)
        x_old = x_new
        if err <= tol:
            break
        
    # Compile DataFrame
    n = len(x_new)
    df = pd.DataFrame(history, columns=[f"x{i+1}" for i in range(n)])
    df["error"] = errors
    df.index.name = "Iteration"
    return df

### Kết quả

In [23]:
#Original matrix Ax=B
A = input_matrix('JCB_input_A.txt', convert_fractions=False)
B = input_matrix('JCB_input_B.txt', convert_fractions=False).flatten() #remove flatten if B is multi-column matrix

print("\nMatrix A:"); output_matrix(A)
print("\nCheck dominace of A:", check_dominance(A));
print("\nMatrix B:"); output_matrix(B)


Matrix A:
  130.75     3.75     5.25     11.25    18.25      6.5     12.0     15.5     2.75     20.0
    -7.0    156.0      4.0       8.0  17.6667  -3.6667     -3.0      4.0      9.0  21.6667
     2.8      4.2     89.0       5.4     10.8     12.6     -7.8      8.0     14.8     13.0
 16.5714  14.8571  25.4286  140.4286  25.7143   2.1429   5.5714  10.5714   5.4286   6.4286
  6.1667    -19.0   9.8333   12.1667    114.0  -6.1667     14.5   3.6667    -10.5 -14.1667
  -0.875   11.125    7.125     6.375   13.375    81.75     -6.0    1.875   -15.75    -8.75
 -7.5714  -1.4286   5.2857    9.8571    -18.0   7.1429  96.8571 -10.1429  -8.8571 -12.8571
    17.2     11.0     15.6       2.8      2.2     14.2     15.6    106.8      5.2      6.4
 15.8333   4.8333  -5.8333       5.5  -9.6667   7.6667   5.6667      9.5  81.1667     -3.0
  0.9167     -2.0  -2.5833    2.9167   3.9167  10.6667   2.1667   9.5833  -3.5833  52.0833

Check dominace of A: 1

Matrix B:
 310.2275
-193.5166
  238.112
-165.0229
 214

In [24]:
#Convert to recursion form x_new = Cx+D
C, D = convert_to_iteration(A, B)

print("\nMatrix C:"); output_matrix(C)
print("\nMatrix D:"); output_matrix(D)


Matrix C:
            0.0 -0.028680688337 -0.040152963671  -0.08604206501 -0.139579349904 -0.049713193117 -0.091778202677 -0.118546845124  -0.02103250478 -0.152963671128
 0.044871794872             0.0 -0.025641025641 -0.051282051282 -0.113248076923  0.023504487179  0.019230769231 -0.025641025641 -0.057692307692 -0.138889102564
-0.031460674157 -0.047191011236             0.0 -0.060674157303 -0.121348314607 -0.141573033708  0.087640449438 -0.089887640449 -0.166292134831  -0.14606741573
-0.118005876296 -0.105798249075 -0.181078498255             0.0 -0.183112984107 -0.015259712053  -0.03967425439 -0.075279537074 -0.038657367516 -0.045778424053
-0.054093859649  0.166666666667 -0.086257017544 -0.106725438596             0.0  0.054093859649 -0.127192982456 -0.032164035088  0.092105263158  0.124269298246
 0.010703363914 -0.136085626911 -0.087155963303 -0.077981651376 -0.163608562691             0.0  0.073394495413 -0.022935779817  0.192660550459  0.107033639144
 0.078170831049  0.0147495640

In [25]:
#Calculate the result
x0 = np.array([0,0,0,0,0,0,0,0,0,0]) #initial value
domiType = check_dominance(A)
eps = None
eta = 1e-6

df_history = fixed_point_matrix_iteration(A, C, D, x0, domiType, eps, eta)
print(df_history)


Shrinking value: 0.892134831461, Additional value: 1.000000000000, Threshold: 0.000000120907
                       x1              x2              x3              x4              x5              x6              x7              x8              x9             x10           error
Iteration                                                                                                                                                                                
0          0.000000000000  0.000000000000  0.000000000000  0.000000000000  0.000000000000  0.000000000000  0.000000000000  0.000000000000  0.000000000000  0.000000000000             NaN
1          2.372676864245 -1.240491025641  2.675415730337 -1.175137400786  1.880890350877  1.328837920489  0.297905884029  2.731573033708 -3.161046340433  3.785967863019  1.000000000000
2          1.209554697354 -1.731908259876  2.067464277633 -2.441607485884  1.565900857901  0.829208380800  1.189771612046  1.785334420966 -3.380198572453  2.749009

In [26]:
solution_series = df_history.filter(regex=r'^x\d+$').iloc[-1]
print("Approximate solution:"),
print(solution_series.to_string())

Approximate solution:
x1     1.520000142154
x2    -1.540000208642
x3     2.500000100122
x4    -2.140000388434
x5     1.499999927330
x6     0.999999842530
x7     0.999999549144
x8     2.000000059962
x9    -3.150000396881
x10    2.999999205458


## 2. Ma trận chéo trội Cột

### Thuật toán

In [27]:
def convert_to_iteration_2(A: np.ndarray, 
                           B: np.ndarray, 
                          ) -> Tuple[np.ndarray, np.ndarray]:
    """
    Prepares iteration y_{k+1} = C y_k + D for column-dominant case.
    """

    # 1. T = diag of 1 / diagonal entries of A
    T = np.diag(1.0 / np.diag(A))

    # 2. C = I - A * T
    n = A.shape[0]
    C = np.eye(n) - A.dot(T)
    
    # 3. D = B
    D = B.copy()

    return T, C, D

In [28]:
def fixed_point_matrix_iteration_2(
    A: np.ndarray,
    T: np.ndarray,
    C: np.ndarray, #main input
    D: np.ndarray,
    x0: Union[np.ndarray, list],
    domiType: int,
    eps: float,
    eta: float
) -> np.ndarray:
    """
    Iterates y_{k+1} = C y_k + D for column-dominant systems.
    
    Parameters:
    - C: (n×n) iteration matrix
    - D: (n,) constant vector
    - y0: initial guess (n,)
    - domiType: dominance type (must be 2 or 3 for column-dominant)
    - eps: relative error tolerance
    
    Uses:
    - vector norm: 1-norm
    - matrix norm: column-sum norm
    - alpha = max|diag(A)| / min|diag(A)| from original A
    
    Prints each iteration's y and error, stops when
      ||y_new - y_old||_1 <= eps * (1 - q) / (alpha * q)
    
    Returns:
    - y_new: approximate solution vector
    """
     # Validate tolerance inputs
    if (eps is None) == (eta is None):
        raise ValueError("Specify exactly one of eps (exact) or eta (relative)")

    # Set norms and α, q based on dominance type
    if domiType in (2,3):
        vec_norm = lambda x: np.sum(np.abs(x))
        q = np.max(np.sum(np.abs(C), axis=0))  # column-sum norm
        diag_vals = np.abs(np.diag(A))         # A should be defined globally
        alpha = np.max(diag_vals) / np.min(diag_vals)
    else:
        raise ValueError("domiType must be 2 for column-dominant iteration")

    # Convergence threshold
    tol = (eps if eps is not None else eta) * (1 - q) / (alpha * q)
    print(f"Shrinking value: {q:.12f}, Additional value: {alpha:.12f}, Threshold: {tol:.12f}")

    # Initialize
    x_old = np.array(x0, dtype=float).flatten()
    T_inv = np.diag(np.diag(A)); y_old = T_inv.dot(x_old)
    history = [np.concatenate([y_old, x_old])]
    errors = [np.nan]

    while True:
        y_new = C.dot(y_old) + D
        x_new = T.dot(y_new)
        if eps is not None:
            err = vec_norm(x_new - x_old)
        else:
            err = vec_norm(x_new - x_old) / vec_norm(x_new)

        # Prepare DataFrame for display
        history.append(np.concatenate([y_new, x_new]))
        errors.append(err)
        y_old = y_new; x_old = x_new;
        if err <= tol:
            break
        
    # Compile DataFrame
    n = len(x_new)
    df = pd.DataFrame(history, columns=[f"y{i+1}" for i in range(n)] + [f"x{i+1}" for i in range(n)])
    df["error"] = errors
    df.index.name = "Iteration"
    return df

### Kết quả

In [29]:
#Original matrix Ax=B
A = input_matrix('JCB_input_A1.txt', convert_fractions=False)
B = input_matrix('JCB_input_B1.txt', convert_fractions=False).flatten() #remove flatten if B is multi-column matrix

print("\nMatrix A:"); output_matrix(A)
print("\nCheck dominace of A:", check_dominance(A));
print("\nMatrix B:"); output_matrix(B)


Matrix A:
 34.4721   7.1502   7.7019   2.1981   8.4253
  3.1972  32.5598   4.5293   9.6084   2.1885
  8.7633   8.9285  26.2682   2.0614   6.7754
  8.4911   6.9724   7.4731  31.1989   2.5773
  7.2342   7.8641   0.2359   8.7465  21.4682

Check dominace of A: 2

Matrix B:
 4.9119
 5.7773
 6.2444
 7.7651
 7.6242


In [30]:
#Convert to recursion form x_new = Cx+D
T, C, D = convert_to_iteration_2(A, B)

print("\nMatrix C:"); output_matrix(C)
print("\nMatrix D:"); output_matrix(D)


Matrix C:
            0.0 -0.219602086008 -0.293202427269 -0.070454407046 -0.392454886763
-0.092747468242             0.0 -0.172425213757 -0.307972396463  -0.10194147623
-0.254214277633 -0.274218514856             0.0 -0.066072842312 -0.315601680625
-0.246318036905 -0.214141364505 -0.284492275832             0.0 -0.120051983865
-0.209856666696 -0.241527896363 -0.008980440228  -0.28034642247             0.0

Matrix D:
 4.9119
 5.7773
 6.2444
 7.7651
 7.6242


In [31]:
#Calculate the result
domiType = check_dominance(A)
x0 = [1,1,1,1,1] #initial value
eps = 1e-6
eta = None

df_history = fixed_point_matrix_iteration_2(A, T, C, D, x0, domiType, eps, eta)
print(df_history)


Shrinking value: 0.949489861731, Additional value: 1.605728472811, Threshold: 0.000000033130
                        y1               y2               y3               y4               y5              x1              x2              x3              x4              x5           error
Iteration                                                                                                                                                                                     
0          34.472100000000  32.559800000000  26.268200000000  31.198900000000  21.468200000000  1.000000000000  1.000000000000  1.000000000000  1.000000000000  1.000000000000             NaN
1         -20.563600000000 -13.746100000000 -20.284200000000 -17.748800000000 -16.456500000000 -0.596528787048 -0.422180111671 -0.772196039318 -0.568891851956 -0.766552389115  8.126349179108
2          21.586863933489  18.325769732746  21.607808567430  23.520227888212  20.417648796270  0.626212616391  0.562834222960  0.822584286987 

In [32]:
solution_series = df_history.filter(regex=r'^x\d+$').iloc[-1]
print("Approximate solution:"),
print(solution_series.to_string())

Approximate solution:
x1    0.026332654664
x2    0.090795087475
x3    0.122265747518
x4    0.172195351847
x5    0.241507609384
